# Deploying and triggering Flex template runs

In [3]:
%cd ../drift_detector

/home/jarekk/repos/drift-monitor/drift_detector


In [4]:
import json
from datetime import datetime
import time

## Build the Flex template docker image

In [5]:
!ls -la

total 40
drwxr-xr-x  4 jarekk jarekk 4096 May 17 17:13 .
drwxr-xr-x 10 jarekk jarekk 4096 May 17 17:03 ..
drwxr-xr-x  3 jarekk jarekk 4096 May 17 17:06 coders
-rw-r--r--  1 jarekk jarekk  895 May 17 17:24 Dockerfile
-rw-r--r--  1 jarekk jarekk  784 May 13 18:26 __init__.py
-rw-r--r--  1 jarekk jarekk 1966 May 15 02:46 metadata.json
-rw-r--r--  1 jarekk jarekk 4491 May 16 15:48 run.py
-rw-r--r--  1 jarekk jarekk  256 May 13 18:26 setup.py
drwxr-xr-x  2 jarekk jarekk 4096 May 17 17:13 utils


In [37]:
template_name = 'drift-detector'

project_id = !(gcloud config get-value core/project)
project_id = project_id[0]
template_image = 'gcr.io/{}/{}:latest'.format(project_id, template_name)

!gcloud builds submit --tag {template_image} .

e-packages (from apache-beam[gcp]<3,>=2.20->tensorflow_data_validation[visualization]==0.22.0) (2.8.1)
  Created wheel for absl-py: filename=absl_py-0.8.1-py3-none-any.whl size=121165 sha256=9a26ae129f5c1a5108c4e3c842099785a23805d0ffbb2cef36cbf44291f59436
  Stored in directory: /root/.cache/pip/wheels/0c/fa/af/c3217c654740d2cffc40ab4eca9da558be4bb94dd593ef0d74
  Created wheel for backcall: filename=backcall-0.1.0-py3-none-any.whl size=10413 sha256=72dd4ace6dd42c90eff4ba5526819a69ab3a16e0e217440e1f2d17408e935c39
  Stored in directory: /root/.cache/pip/wheels/cb/6f/5c/9b66a87adeb46fed2babb14407bae1271f5e41b6b34b642411
Successfully built absl-py backcall
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.9.0
    Uninstalling absl-py-0.9.0:
      Successfully uninstalled absl-py-0.9.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 0.15.1
    Uninstalling pyarrow-0.15.1:
      Successfully uninstalled pyarrow-0.15.1
You should consider upgra

## Deploy the Flex template using `gcloud`

In [38]:
flex_templates_location = 'gs://mlops-dev-workspace/flex-templates'
template_path = '{}/{}.json'.format(flex_templates_location, template_name)
metadata_file = 'metadata.json'

!gcloud beta dataflow flex-template build {template_path} \
  --image {template_image} \
  --sdk-language "PYTHON" \
  --metadata-file {metadata_file}

Successfully saved container spec in flex template file.
Template File GCS Location: gs://mlops-dev-workspace/flex-templates/drift-detector.json
Container Spec:

{
    "image": "gcr.io/mlops-dev-env/drift-detector:latest",
    "metadata": {
        "description": "Data drift detector Python flex template.",
        "name": "Data drift detector Python flex template",
        "parameters": [
            {
                "helpText": "A full name of the BQ request-response log table",
                "label": "Request response log table.",
                "name": "request_response_log_table",
                "regexes": [
                    "[-_.a-zA-Z0-9]+"
                ]
            },
            {
                "helpText": "A type of instances in request_response log_table",
                "label": "Instance type.",
                "name": "instance_type",
                "regexes": [
                    "LIST|OBJECT"
                ]
            },
            {
              

## Trigger a run of the template using REST API

In [39]:
service_url = 'https://dataflow.googleapis.com/v1b3/projects/{}/locations/us-central1/flexTemplates:launch'.format(project_id)
headers_content = 'Content-Type: application/json'
access_token = !(gcloud auth print-access-token) 
headers_auth = 'Authorization: Bearer {}'.format(access_token[0])
parameters = {
    'request_response_log_table': 'mlops-dev-env.data_validation.covertype_classifier_logs_tf',
    'instance_type': 'OBJECT',
    'start_time': '2020-05-15T00:15:00',
    'end_time': '2020-05-15T05:51:00',
    'output_path': 'gs://mlops-dev-workspace/drift_monitor/output/tf',
    'schema_file': 'gs://mlops-dev-workspace/drift_monitor/schema/schema.pbtxt',
    'setup_file': './setup.py',

}
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
body = {
    'launch_parameter': 
         {
             'jobName': job_name,
             'parameters' : parameters,
             'containerSpecGcsPath': template_path
         }}

json_body = json.dumps(body)

In [40]:
!curl -X POST \
  "{service_url}" \
  -H "{headers_content}" \
  -H "{headers_auth}" \
  -d '{json_body}'

{
  "job": {
    "id": "2020-05-17_12_21_11-2472520780605503114",
    "projectId": "mlops-dev-env",
    "name": "data-drift-20200517-192110",
    "currentStateTime": "1970-01-01T00:00:00Z",
    "createTime": "2020-05-17T19:21:12.702168Z",
    "location": "us-central1",
    "startTime": "2020-05-17T19:21:12.702168Z"
  }
}


## Trigger a run of the template using `gcloud`

In [ ]:
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
g_parameters = ','.join(['{}={}'.format(key,value) for key, value in parameters.items()])

!gcloud beta dataflow flex-template run {job_name} \
--template-file-gcs-location {template_path} \
--parameters {g_parameters}